# ARR for Data Profressionals

## Preamble

<div class="alert alert-block alert-info">
Building ARR is simple, but that doesn't make it easy. In my experience, the definitions are where the problems in the metric take root. Depending on who you talk to or where you work, definitions can vary.<b> The important thing is consistency.</b> ARR is shiny and will get a lot of attention, so you better know what you're doing.</div>

## Introduction

The goal of this write up is to share my knowledge on building ARR datasets. I've built and scaled these datasets for companies less than $\text{\$30M}$ and more than $\text{\$1B}$ ARR. All the way from ingestion of source systems to desigining the visuals.

I'll explain how it all works, show some mistakes I've made, and make recommendations. Whether you are 12 sheets deep in excel or lost in spark configurations, as long as you are dealing with ARR on the data side, this is written for you. I'll start with what ARR is and how to calculate it. Then I'll dive into the customer cube. Build on top of our cube with all the different cuts and measures. Lastly, I'll throw in some forecasting.

My goal is to generalize the code as much as possible. So, python with the usual data stack (pandas, numpy, etc.).

In terms of Finance history, ARR is new and not to be confused with Revenue Recognition. ARR is Non-[GAAP](https://en.wikipedia.org/wiki/Accounting_standard). That distinction, I think, adds to a lot of chatter. [This article](https://sensiba.com/resources/insights/what-you-need-to-know-about-annual-recurring-revenue-and-gaap-revenue-recognition/) explains the difference well. Which means, everything I explain is not a law or notable standard. I'm just putting on paper everything I've been taught and been able to soak up from some pretty smart people.

## Table of Contents

- [What is ARR? Part 1](2-what-is-arr-part-1.ipynb)
- [What is ARR? Part 2](3-what-is-arr-part-2.ipynb)
- [What is ARR? Part 3](4-what-is-arr-part-3.ipynb)
- [The Customer Cube](5-the-customer-cube.ipynb)

##### Packages used in Notebooks

In [1]:
# Installing libraries used in entire notebook
# Written w/ Python3.12
%pip install plotly pandas Jinja2 importnb openpyxl

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/19.1 MB 4.2 MB/s eta 0:00:05
   --------- ------------------------------ 4.5/19.1 MB 12.8 MB/s eta 0:00:02
   ------------------- -------------------- 9.2/19.1 MB 16.3 MB/s eta 0:00:01
   ----------------------------- ---------- 14.2/19.1 MB 18.5 MB/s eta 0:00:01
   ---------------------------------------  18.9/19.1 MB 19.5 MB/s eta 0:00:01
   ---------------------------------------  18.9/19.1 MB 19.5 MB/s eta 0:00:01
   ---------------------------------------  18.9/19.1 MB 19.5 MB/s eta 0:00:01
   ---------------------------------------  18.9/19.1 MB 19.5 MB/s eta 0:00:01
   ---------------------------------------  18.9/19.1 MB 19.5 MB/s eta 0:00:01
   ---------------------------------------  18.9/19.1 MB 19.5 MB/s eta 0:00


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\CStallings\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
          